# PART 1: CLASSIFICATION OF ENTITIES

In [134]:
# Import libraries
import pandas as pd
import numpy as np
import re

In [267]:
# Store the dataframe in a variable
companies_original = pd.read_excel('Data_Science_Internship_Assignment.xlsx', sheet_name = 'Data')

# Create a copy, we will work with the copy
companies = companies_original.copy()

In [268]:
# Check how the dataframe looks like
companies.head(5)

,NAME,WEBSITE,TAGLINE,HQ REGION,HQ COUNTRY,HQ CITY,TAGS,LAUNCH DATE,GROWTH STAGE,LINKEDIN,TYPE
0,63336,http://63336.com,Ai-enabled q&a service that answers to various...,Europe,United Kingdom,London,mobile,"2002, September",late growth stage,NaN,NaN
1,@Futsal,http://futsaluk.net,Educational courses through the medium of spor...,Europe,United Kingdom,Birmingham,NaN,2008,early growth stage,https://www.linkedin.com/company/-futsal-group...,NaN
2,#5 Magazine,http://5mag.co,Multi-platform digital lifestyle magazines abo...,Europe,United Kingdom,London,publishing;branding;media;platform;entertainment,2007,NaN,NaN,NaN
3,03Numbers,http://planet-numbers.co.uk/numbers/0333,Planet Numbers are the leading provider of 03 ...,Europe,United Kingdom,NaN,NaN,"2008, September",early growth stage,http://www.linkedin.com/company/planet-numbers,NaN
4,077football News & Media,http://077football.com,"077Football - the deep-rooted, hyperlocal foot...",Europe,United Kingdom,Cambridge,sport;advertising;football;network;game develo...,2009,late growth stage,https://www.linkedin.com/company/077football-n...,NaN


In [269]:
# Check for missing values
companies.isna().sum()

NAME                0
WEBSITE             0
TAGLINE           129
HQ REGION           0
HQ COUNTRY          0
HQ CITY           674
TAGS             1989
LAUNCH DATE         0
GROWTH STAGE     2998
LINKEDIN         2986
TYPE            11582
dtype: int64

# Clean years column

In [270]:
# Convert every value in LAUNCH DATE to only numbers
years = []
for date in companies['LAUNCH DATE']:
    year = int(re.sub(r'\D', '', str(date)))
    years.append(year)

companies['LAUNCH DATE'] = years

# Unclassified companies

In [271]:
# All companies without a tagline.
unclassified_df = companies[companies['TAGLINE'].isna()]

# Give the selected companies a tag
companies.loc[unclassified_df.index, 'TYPE'] = 'unclassified'

# Schools and Universities

In [272]:
# Empty list
schools = []

# Find all schools
for name in companies['NAME']:
    x = re.findall(r'^.*School.*$', str(name))
    if x != []:
        y = str(x[0]).split()
        if 'Computing' in y:
            continue
        elif 'Holding' in y:
            continue
        elif 'Interactive' in y:
            continue
        elif 'Driving' in y:
            continue
        elif 'Radio' in y:
            continue
        else:
            schools.append(x[0])

In [273]:
# All schools in a dataframe
schools_df = companies[companies['NAME'].isin(schools)]

# Give the selected companies a tag
companies.loc[schools_df.index, 'TYPE'] = 'university/school'

# Find non-profit organisations

In [274]:
# Empty lists
npo = []
charity = []
foundation = []

# For loops to find non-profit organisations
for tagline in companies['TAGLINE']:
    x = re.findall(r'^.*no.*profit[^a].*$', str(tagline).lower())
    if x != []:
        y = str(x[0]).split()
        if 'solution' in y:
            continue
        else:      
            npo.append(x[0])

for tagline in companies['TAGLINE']:
    x = re.findall(r'^.*charity[^c].*$', str(tagline).lower())
    if x != []:
        y = str(x[0]).split()
        if 'computing' in y:
            continue
        elif '20+' in y:
            continue
        else:        
            charity.append(x[0])
                     
for tagline in companies['TAGLINE']:
    x = re.findall(r'^.*\sfoundation\s.*$', str(tagline).lower())
    if x != []:
        y = str(x[0]).split()
        if 'technical' in y:
            continue
        elif 'architects,' in y:
            continue
        elif 'software' in y:
            continue
        else:     
            foundation.append(x[0])

# Create a set in order to remove duplicates
non_profit = set(npo + charity + foundation)

# Store it in a list again
non_profit = list(non_profit)

In [275]:
# Give the selected companies a tag
non_profit_df = companies
non_profit_df['TAGLINE'] = companies['TAGLINE'].map(lambda x: str(x).lower())
non_profit_df = non_profit_df[non_profit_df['TAGLINE'].isin(non_profit)]
companies.loc[non_profit_df.index, 'TYPE'] = 'non-profit'

# Seperate startups and mature companies

In [276]:
# Companies without label
other_companies = companies[companies['TYPE'].isna()]

# Separate remaining companies based on LAUNCH DATE
# Select startups (companies launched in 1990 or later)
startups = other_companies[other_companies['LAUNCH DATE'] >= 1990]
companies.loc[startups.index, 'TYPE'] = 'startup'

# Select mature companies (companies launched before 1990)
mature = other_companies[other_companies['LAUNCH DATE'] < 1990]
companies.loc[mature.index, 'TYPE'] = 'mature company'

In [279]:
# Check if every company has a tag
companies.isna().sum()['TYPE'] == 0

True

# Number of entities per type

In [323]:
# Create dataframe with the number of entities per type
entity_count = pd.DataFrame(companies['TYPE'].value_counts())
entity_count.columns = ['Count']

# PART 2: SCRAPING

In [1]:
# Import libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import requests
import urllib
import lxml
import time

In [2]:
# Store url and path to chromedriver in a variable
path = '../../ironhack/chromedriver'
url = 'https://www.ycombinator.com/companies/'

# Set options
options = Options()
options.add_argument('permissions.default.image')
options.add_argument('dom.ipc.plugins.enabled.libflashplayer.so')

In [3]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

In [4]:
def extract_info(url):
    # Setup the driver
    driver = webdriver.Chrome(options=options ,executable_path=path)
    # Let driver wait before throwing an exception
    driver.implicitly_wait(30)
    # Open the page
    driver.get(url)
    # Scroll and timeout
    scroll(driver, 5)
    # Create the soup
    soup = BeautifulSoup(driver.page_source, 'lxml')
    # Close the driver
    driver.close()

    # Empty lists to store the info
    comp_names = []
    comp_places = []
    comp_descriptions = []

    # Looping through all the elements in the page source
    for name in soup.select('span.SharedDirectory-module__coName___gbFfW'):
        comp_names.append(name.get_text())
    
    for place in soup.select(' div.right > div:nth-child(1) > span:nth-child(2)'):
        comp_places.append(place.get_text())
        
    for desc in soup.select(' div.right > div:nth-child(2)'):
        comp_descriptions.append(desc.get_text())


    return comp_names, comp_places, comp_descriptions

# Store the extracted info in variables
names, places, descriptions = extract_info(url)

In [7]:
# Empty lists
city = []
state = []
country = []

# Separate the city, state and country and put them in different lists
for place in places:
    splitted = place.split(', ')
    if len(splitted) == 4:
        city.append(splitted[1])
        state.append(splitted[2])
        country.append(splitted[3])
  
    elif len(splitted) == 3:
        city.append(splitted[0])
        state.append(splitted[1])
        country.append(splitted[2])
        
    elif len(splitted) == 2:
        city.append(splitted[0])
        state.append(np.nan)
        country.append(splitted[1])
        
    elif len(splitted) == 1:
        city.append(np.nan)
        state.append(np.nan)
        country.append(splitted)
    else:
        city.append(np.nan)
        state.append(np.nan)
        country.append(np.nan)
    

In [8]:
# Create a dataframe with all the variables
startup_directory =  pd.DataFrame({'Company': names,
                                   'City': city,
                                   'State': state,
                                   'Country': country,
                                   'Description': descriptions
})

# Write dataframes in excel

In [291]:
# Import library
import xlsxwriter

In [308]:
# Path to file 
assignment_file = '/Users/nick/documents/Sollicitaties/dealroom_assignment/Assignment_adjusted.xlsx'

In [325]:
# Write in the excel file and save it
writer = pd.ExcelWriter(assignment_file, engine='xlsxwriter')

companies.to_excel(writer, sheet_name = 'Data')
entity_count.to_excel(writer, sheet_name = 'Count')
startup_directory.to_excel(writer, sheet_name = 'Scraping results')

writer.save()